## demo for parsing

In [ ]:
%load_ext autoreload
%autoreload 2
from tree_sitter_languages import get_parser
parser = get_parser("java")

with open("grammar_test.java", "r") as file:
    source_code = file.read()
print(source_code)

import utils
tree = parser.parse(bytes(source_code, "utf8"))
print(utils.format_code(tree.root_node.sexp()))

## Parse all solutions

In [5]:
# get program model from a java file
def getProgram(file_path):
    with open(file_path, "r") as file:
        source_code = file.read()
        utils.source_code_line = source_code.split("\n")
    tree = parser.parse(bytes(source_code, "utf8"))
    return utils.visit_program(tree.root_node)

# print program as a java file
def printJava(program):
    return utils.addIndentation(program.toString())

# validate the parser and the java generation
def validateJava(file_path):
    return_value = True
    try:
        program = getProgram(file_path)
        javaCode= printJava(program)
        tree = parser.parse(bytes(javaCode, "utf8"))
    except Exception as e:
        print(f"\nError in parsing {file_path}")
        print(e)
        return_value = False
    finally:
        return return_value
    
test_filepath = "grammar_test.java"
program = getProgram(test_filepath)
with open("test.java", "w") as file:
    file.write(printJava(program))
validateJava("test.java")

True

In [ ]:
import os
import subprocess
# Path to the solutions folder
folder_path = "solutions"

# Get the list of Java code files in the folder
java_files = [file for file in os.listdir(folder_path) if file.endswith(".java")]

java_files.sort()
error_cnt=0
# Loop through each Java code file
for file_name in java_files[:]:
    file_path = os.path.join(folder_path, file_name)
    
    # using the parser to validate the java file
    if not validateJava(file_path):
        error_cnt+=1

    # using javac to validate the java file
    # try:
    #     with open("test.java", "w") as file:
    #         file.write(printJava(getProgram(file_path)))
    # except Exception as e:
    #     print(f"\nError in parsing {file_path}")
    #     print(e)
    #     error_cnt+=1
    #     continue
    # res = subprocess.run(
    #             ["javac", "-d", "tmp", file_path],
    #             stdout=subprocess.PIPE,
    #             stderr=subprocess.PIPE,
    #             timeout=60,
    #         )
    # if res.returncode != 0:
    #     print(f"\nError in parsing {file_path}")
    #     print(res.stderr.decode("utf-8"))
    #     error_cnt+=1
print(f"Total errors: {error_cnt} out of {len(java_files)} files.")

## Translating Program Model into Coq Proof

In [11]:
# coq proof test
def validateCoqProof(file_path):
    program=getProgram(file_path)
    coqProof=program.toCoq().toString()
    prefix="""From PLF Require Import Syntax.
Open Scope string_scope.

Example prog_well_typed : exists p, program_well_typed p.
Proof with (simpl;try(reflexivity)).
unfold program_well_typed.
eexists.
eexists."""
    suffix="""Defined.
Definition prog := the_exists_term (prog_well_typed).
Print prog."""
    coqProof=f"{prefix}\n{coqProof}\n{suffix}"
    with open("../coq_code/test.v", "w") as file:
        file.write(coqProof)
    res = subprocess.run(
                # coqc -Q ../coq_code PLF ../coq_code/test.v
                ["coqc", "-Q","../coq_code", "PLF", "../coq_code/test.v"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=60,
            )
    print(res)

validateCoqProof("test.java")

CompletedProcess(args=['coqc', '../demo/test.v'], returncode=1, stdout=b'', stderr=b'File "../demo/test.v", line 1, characters 0-31:\nError: Cannot find a physical path bound to logical path\nSyntax with prefix PLF.\n\n')
